In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Preparing Data

In [2]:
import torchtext
from torchtext.data import Field, LabelField, BucketIterator

# Set `batch_first=True` in the `Field`.
TEXT = Field(tokenize='spacy', tokenizer_language='en_core_web_sm', include_lengths=True, batch_first=True)
LABEL = LabelField()

# `filter_pred`: Use only examples for which `filter_pred(example)` is True.
# Note: Some sentences have lengths shorter than CNN kernel size...
train_data, test_data = torchtext.datasets.TREC.splits(TEXT, LABEL, fine_grained=False, root="../assets/data", 
                                                       filter_pred=lambda ex: len(ex.text)>=4)
train_data, valid_data = train_data.split()

In [3]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE, 
                 vectors="glove.6B.100d", vectors_cache="../assets/vector_cache", 
                 unk_init=torch.Tensor.normal_)

LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)

defaultdict(None, {'HUM': 0, 'ENTY': 1, 'DESC': 2, 'NUM': 3, 'LOC': 4, 'ABBR': 5})


In [4]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, device=device)

In [5]:
for batch in train_iterator:
    break
text, text_lens = batch.text
print(text)
print(text_lens)

tensor([[ 309,  178,   14,  ...,    1,    1,    1],
        [   4, 1506, 2127,  ...,    1,    1,    1],
        [   4,    5,    3,  ...,    1,    1,    1],
        ...,
        [  15,   92,    3,  ...,    1,    1,    1],
        [   4,   21, 4692,  ...,    1,    1,    1],
        [  54,    8,   33,  ...,    1,    1,    1]], device='cuda:0')
tensor([14, 12,  9, 11, 10, 10,  8,  7,  9,  5,  8,  8,  9,  7, 12, 13,  8, 12,
        14,  8, 12,  6,  7, 10, 11, 10, 16,  8,  9, 15,  7, 12,  9, 15,  9,  7,
         7, 13, 23,  7,  8,  7, 11, 11,  7, 19, 11, 10,  9, 11, 13,  7, 13,  8,
         9,  6,  9, 11, 13,  5,  8,  5,  7, 12], device='cuda:0')


# Build Model

In [6]:
class Classifier1D(nn.Module):
    def __init__(self, in_dim, emb_dim, n_filters, filter_sizes, out_dim, dropout, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        # (2) Regard different embedding dimensions as different channels. 
        # Thus, it reduces to an 1D convolution. 
        self.convs = nn.ModuleList([
            nn.Conv1d(emb_dim, n_filters, kernel_size=fs)
            for fs in filter_sizes])

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(n_filters*len(filter_sizes), out_dim)

    def forward(self, text, text_lens):
        # `text_lens` would be useful when mean-pooling applied. 
        # text/mask: (batch, step)
        mask = text.eq(self.emb.padding_idx)
        # embedded: (batch, step, emb)
        embedded = self.emb(text)
        
        pooled = []
        for conv_i in self.convs:
            # Note: The kernel size may exceed the sequence length, which leads to invalid output. 
            # conved_i: (batch, channel=n_filters, step-(kernel_size-1))
            conved_i = F.relu(conv_i(embedded.permute(0, 2, 1)))
            # mask_i: (batch, 1, step-(kernel_size-1))
            mask_i = mask[:, (conv_i.kernel_size[0]-1):].unsqueeze(1)
            # pooled_i: (batch, n_filters)
            pooled_i, _ = conved_i.masked_fill(mask_i, -np.inf).max(dim=-1)
            pooled.append(pooled_i)

        concated = self.dropout(torch.cat(pooled, dim=-1))
        return self.fc(concated)

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

IN_DIM = len(TEXT.vocab)
EMB_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2, 3, 4]
OUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

classifier1d = Classifier1D(IN_DIM, EMB_DIM, N_FILTERS, FILTER_SIZES, OUT_DIM, 
                           DROPOUT, PAD_IDX).to(device)
print(count_parameters(classifier1d))

classifier = classifier1d

842106


In [8]:
# Initialize Embeddings with Pre-Trained Vectors
classifier.emb.weight.data.copy_(TEXT.vocab.vectors)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

classifier.emb.weight.data[UNK_IDX].zero_()
classifier.emb.weight.data[PAD_IDX].zero_()

print(classifier.emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.1638,  0.6046,  1.0789, -1.2731, -0.7759,  0.3997,  0.3817, -0.1759],
        [-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344],
        [-1.3771, -0.9446,  1.1019,  1.7499, -1.1453,  0.6823,  0.8769,  0.8717]],
       device='cuda:0', grad_fn=<SliceBackward>)


# Train Model

In [9]:
optimizer = optim.Adam(classifier.parameters())
loss_func = nn.CrossEntropyLoss().to(device)

In [10]:
def train_epoch(classifier, iterator, optimizer, loss_func):
    classifier.train()
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        # Forward pass
        text, text_lens = batch.text
        preds = classifier(text, text_lens)
        # Calculate loss
        loss = loss_func(preds, batch.label)
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        # Update weights
        optimizer.step()
        # Accumulate loss and acc
        epoch_loss += loss.item()
        epoch_acc += (preds.argmax(-1) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def eval_epoch(classifier, iterator, loss_func):
    classifier.eval()
    epoch_loss = 0
    epoch_acc = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            text, text_lens = batch.text
            preds = classifier(text, text_lens)
            # Calculate loss
            loss = loss_func(preds, batch.label)
            # Accumulate loss and acc
            epoch_loss += loss.item()
            epoch_acc += (preds.argmax(-1) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [11]:
import time
N_EPOCHS = 10
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss, train_acc = train_epoch(classifier, train_iterator, optimizer, loss_func)
    valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = int(epoch_secs // 60), int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(classifier.state_dict(), "models/tut5-model.pt")
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 1.285 | Train Acc: 49.46%
	 Val. Loss: 0.944 |  Val. Acc: 66.45%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.845 | Train Acc: 69.38%
	 Val. Loss: 0.740 |  Val. Acc: 75.09%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.636 | Train Acc: 78.31%
	 Val. Loss: 0.616 |  Val. Acc: 78.92%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.487 | Train Acc: 83.98%
	 Val. Loss: 0.544 |  Val. Acc: 80.60%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.365 | Train Acc: 88.72%
	 Val. Loss: 0.491 |  Val. Acc: 82.82%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.273 | Train Acc: 91.41%
	 Val. Loss: 0.464 |  Val. Acc: 83.54%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.218 | Train Acc: 93.87%
	 Val. Loss: 0.440 |  Val. Acc: 84.15%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.168 | Train Acc: 95.72%
	 Val. Loss: 0.430 |  Val. Acc: 85.16%
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.124 | Train Acc: 96.91%
	 Val. Loss: 0.426 |  Val. Acc: 84.62%
Epoch: 10 | Epoch Time: 0m 0

In [12]:
classifier.load_state_dict(torch.load("models/tut5-model.pt"))

valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
test_loss, test_acc = eval_epoch(classifier, test_iterator, loss_func)

print(f'Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Val. Loss: 0.419 | Val. Acc: 85.33%
Test Loss: 0.349 | Test Acc: 87.77%


# Check Embeddings
* The Embeddings of `<unk>` and `<pad>` tokens
    * Because the `padding_idx` has been passed to `nn.Embedding`, so the `<pad>` embedding will remain zeros throughout training.  
    * While the `<unk>` embedding will be learned.

In [13]:
print(classifier.emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.2182,  0.6441,  1.0855, -1.2012, -0.7938,  0.4054,  0.3634, -0.1556],
        [-0.0208, -0.3146,  0.7318, -0.4200,  0.0396,  0.0428, -0.4307,  0.2997],
        [-1.3390, -0.9507,  1.0864,  1.7516, -1.1326,  0.6453,  0.8339,  0.8543]],
       device='cuda:0', grad_fn=<SliceBackward>)
